In [ ]:
from sklearn.datasets import fetch_20newsgroups
import spacy
import pandas as pd
import torch
import seaborn as sns
import argparse
from fastprogress import progress_bar
import pickle
import numpy as np
import os
import random

In [ ]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
df.columns = ['text', 'target']
texts = [' '.join(text.strip().split()) for text in df['text'].tolist()]

df['text'] = texts
df['text'].replace('', np.nan, inplace=True)
df['text'].replace('--', np.nan, inplace=True)
df.dropna(subset=['text'], inplace=True)

In [ ]:
nlp = spacy.load('en_trf_bertbaseuncased_lg')

In [ ]:
labels = df['target'].tolist() 
texts = df['text'].tolist()

In [ ]:
mini_newsdict = {i:random.sample(list(np.where((np.array(labels) == i) == True )[0]),30) for i in range(20)}

In [ ]:
mini_text_list = []
mini_label_list = []
for i in range(20):
    id_list_ = mini_newsdict[i]
    for idx in id_list_:
        mini_label_list.append(i)
        mini_text_list.append(texts[idx])
        
mini_df = pd.DataFrame()
mini_df['text'] = mini_text_list
mini_df['target'] = mini_label_list
labels_mini = mini_df['target'].tolist()
texts_mini = mini_df['text'].tolist()
text_vectors = np.zeros((len(mini_df), 768))
labels_vectors = []

In [ ]:
#text_vectors = np.zeros((len(mini_df), 768))
#labels_vectors = []
for i in progress_bar(range(len(texts_mini))):
    label = labels_mini[i]
    text = texts_mini[i]
    try:
        text_vectors[i,:] = nlp(text.strip()).vector
        labels_vectors.append(label)
    except:
        print(text)

In [ ]:
pickle.dump(text_vectors, open('mini_newsgroup_vectors.pkl', "wb" ))
pickle.dump(mini_df, open('mini_newsgroup_data.pkl', "wb" ))